# Modeling For-Loop

## Low level model: head, tail and recursion

In [28]:
from typing import TypeVar, Generic, Callable, Iterable, Generator, List
from __future__ import annotations

A=TypeVar('A')
B=TypeVar('B')

### Case 1: Action only depends on the current element in the container

In [13]:
def htr1(f:Callable[[A],B],v:Iterable[A])->Generator[B,None,None]:
    match v:
        case []: return
        case x,*xs:
            r = f(x)
            yield r
            yield from htr1(f,xs)

In [14]:
list(htr1(float,[1,2]))

[1.0, 2.0]

## Low level model: generator comprehension

In [15]:
def gc1(f:Callable[[A],B],v:Iterable[A])->Generator[B,None,None]:
    return (f(x) for x in v)

## High level models

### Functor

In [77]:
from functools import partial
class Functor(Generic[A]):
    pass

Since there are not abstract base class for iterable and the generic base class is only for type hint. So we need to implement each instance of Functor for each concrete iterable class.

In [105]:
class ListFunctor(list[A],Functor[A]):
    def fmap(self,f:Callable[[A],B])->Functor[B]:
        return ListFunctor((f(x) for x in self))
    
    def const(a,b): return a

    def mrb(self,a): return self.fmap(partial(ListFunctor.const,a))

#### `fmap`

In [106]:
a=ListFunctor([1,2,3,4])
a.fmap(float)

[1.0, 2.0, 3.0, 4.0]

In [87]:
a.fmap((lambda x: x+1)).fmap(float).fmap(abs).fmap(print)

2.0
3.0
4.0
5.0


[None, None, None, None]

#### `map-replaced by`

In [107]:
a.mrb([1,2])

[[1, 2], [1, 2], [1, 2], [1, 2]]

In [1]:
import numpy as np

class InfoArray(np.ndarray):

    def __new__(subtype, shape, dtype=float, buffer=None, offset=0,
                strides=None, order=None, info=None):
        # Create the ndarray instance of our type, given the usual
        # ndarray input arguments.  This will call the standard
        # ndarray constructor, but return an object of our type.
        # It also triggers a call to InfoArray.__array_finalize__
        obj = super().__new__(subtype, shape, dtype,
                              buffer, offset, strides, order)
        # set the new 'info' attribute to the value passed
        obj.info = info
        # Finally, we must return the newly created object:
        return obj

    def __array_finalize__(self, obj):
        # ``self`` is a new object resulting from
        # ndarray.__new__(InfoArray, ...), therefore it only has
        # attributes that the ndarray.__new__ constructor gave it -
        # i.e. those of a standard ndarray.
        #
        # We could have got to the ndarray.__new__ call in 3 ways:
        # From an explicit constructor - e.g. InfoArray():
        #    obj is None
        #    (we're in the middle of the InfoArray.__new__
        #    constructor, and self.info will be set when we return to
        #    InfoArray.__new__)
        if obj is None: return
        # From view casting - e.g arr.view(InfoArray):
        #    obj is arr
        #    (type(obj) can be InfoArray)
        # From new-from-template - e.g infoarr[:3]
        #    type(obj) is InfoArray
        #
        # Note that it is here, rather than in the __new__ method,
        # that we set the default value for 'info', because this
        # method sees all creation of default objects - with the
        # InfoArray.__new__ constructor, but also with
        # arr.view(InfoArray).
        self.info = getattr(obj, 'info', None)
        # We do not need to return anything